From `https://www.tensorflow.org/tutorials/keras/keras_tuner`

In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
!pip install keras-tuner

  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78936 sha256=7c48f559dd35182f8ad772c4bdef8fd30fdbf5b2e24c23a726edd970a2d3f0a5
  Stored in directory: c:\users\ericw\appdata\local\pip\cache\wheels\78\e2\80\7fe373cad54ad22b06d0d6204cbc29cead9e69bb2680327775
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=445f6917840bd70cba0d7b1c25f1f0c29625ef71d42ff487eb9ed4d225fdc6c5
  Stored in directory: c:\users\ericw\appdata\local\pip\cache\wheels\56\b0\fe\4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15355 sha256=0a5c172c519e3e799cd23d133403a45fab9f75ed66a9067b5679f298e6ade0d5
  Stored in directory: c:\users\ericw\appdata\local\pip\cache\wheels\ba\ad\c8\2d98360791161cd3db6daf6b5e730f34021fc9367d5879f497
Successfully built keras-tuner future terminaltables


In [4]:
import kerastuner as kt

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 1s 0us/step


In [6]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [8]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 27s]
val_accuracy: 0.856083333492279

Best val_accuracy So Far: 0.8851666450500488
Total elapsed time: 00h 06m 27s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4948 - accuracy: 0.8247 - val_loss: 0.4056 - val_accuracy: 0.8545
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3711 - accuracy: 0.8637 - val_loss: 0.3750 - val_accuracy: 0.8652
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3312 - accuracy: 0.8786 - val_loss: 0.3570 - val_accuracy: 0.8705
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3083 - accuracy: 0.8860 - val_loss: 0.3333 - val_accuracy: 0.8784
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2889 - accuracy: 0.8937 - val_loss: 0.3321 - val_accuracy: 0.8802
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2725 - accuracy: 0.8981 - val_loss: 0.3241 - val_accuracy: 0.8823
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2567 - accuracy: 0.9042 - val_loss: 0.3282 - val_accuracy:

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/49
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4954 - accuracy: 0.8247 - val_loss: 0.4558 - val_accuracy: 0.8246
Epoch 2/49
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3719 - accuracy: 0.8653 - val_loss: 0.3912 - val_accuracy: 0.8543
Epoch 3/49
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3315 - accuracy: 0.8779 - val_loss: 0.3561 - val_accuracy: 0.8758
Epoch 4/49
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3026 - accuracy: 0.8885 - val_loss: 0.3489 - val_accuracy: 0.8688
Epoch 5/49
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2846 - accuracy: 0.8937 - val_loss: 0.3470 - val_accuracy: 0.8770
Epoch 6/49
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2713 - accuracy: 0.8981 - val_loss: 0.3137 - val_accuracy: 0.8867
Epoch 7/49
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2567 - accuracy: 0.9042 - val_loss: 0.3569 - val_accuracy:

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5958 - accuracy: 0.8835
[test loss, test accuracy]: [0.5958139896392822, 0.8834999799728394]
